In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from matplotlib import pyplot as plt
import scipy
import scipy.optimize

In [2]:
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", 100)
pd.set_option("display.max_colwidth", 100)
pd.options.display.float_format = "{: ,.5f}".format

# data

In [3]:
df_events = pd.read_csv("data/events.csv").sort_values(
    [
        "game_id",
        "minute",
    ],
    ignore_index=True,
)
df_games = pd.read_csv("data/games.csv").assign(date=lambda x: pd.to_datetime(x.date))

In [4]:
df = (
    df_games.merge(df_events, how="outer", on=["game_id"])
    .sort_values(by=["game_id", "minute"])
    .query(
        "competition == 'Premier League, England' & '2018-08-01' < date < '2019-08-01'"
    )
    .reset_index(drop=True)
)

In [5]:
list_game_score_minutes = list()

game_id = 0


for i, row in df.iterrows():

    if row.game_id != game_id:
        if game_id != 0:  # Changing match
            # No goal until end-of-match of previous game
            dict_game_score_minutes = dict(
                game_id=game_id,
                home_score=home_score,
                away_score=away_score,
                start_minute=last_minute,
                end_minute=90,
                side="both",
                goal=0,
            )
            list_game_score_minutes.append(dict_game_score_minutes)

        game_id = row.game_id
        home_score, away_score = 0, 0
        last_minute = 0

    if np.isnan(row.minute):  # Match finished 0-0 with no red cards and no goals
        dict_game_score_minutes = dict(
            game_id=game_id,
            home_score=0,
            away_score=0,
            start_minute=0,
            end_minute=90,
            side="both",
            goal=0,
        )
        list_game_score_minutes.append(dict_game_score_minutes)
        continue

    if row.type == "goal":
        # Waiting for the goal
        dict_game_score_minutes = dict(
            game_id=game_id,
            home_score=home_score,
            away_score=away_score,
            start_minute=last_minute,
            end_minute=row.minute,
            side="both",
            goal=0,
        )
        list_game_score_minutes.append(dict_game_score_minutes)
        last_minute = row.minute

        # Scoring the goal
        dict_game_score_minutes = dict(
            game_id=game_id,
            home_score=home_score,
            away_score=away_score,
            start_minute=row.minute,
            end_minute=row.minute,
            side=row.side,
            goal=1,
        )
        list_game_score_minutes.append(dict_game_score_minutes)

        if row.side == "away":
            away_score += 1

        elif row.side == "home":
            home_score += 1

    if i == len(df) - 1:
        # No goal until end-of-match
        dict_game_score_minutes = dict(
            game_id=game_id,
            home_score=home_score,
            away_score=away_score,
            start_minute=last_minute,
            end_minute=90,
            side="both",
            goal=0,
        )
        list_game_score_minutes.append(dict_game_score_minutes)

In [6]:
df_goal_events = pd.DataFrame(list_game_score_minutes).merge(
    df[["game_id", "home_team", "away_team"]].drop_duplicates(ignore_index=True),
    how="left",
    on=["game_id"],
)

In [7]:
df_goal_events

,game_id,home_score,away_score,start_minute,end_minute,side,goal,home_team,away_team
0,68,0,0,0.00000,3.00000,both,0,Manchester United,Leicester City
1,68,0,0,3.00000,3.00000,home,1,Manchester United,Leicester City
2,68,1,0,3.00000,83.00000,both,0,Manchester United,Leicester City
3,68,1,0,83.00000,83.00000,home,1,Manchester United,Leicester City
4,68,2,0,83.00000,90.00000,both,0,Manchester United,Leicester City
...,...,...,...,...,...,...,...,...,...
2539,4071,1,2,46.00000,71.00000,both,0,Watford,West Ham United
2540,4071,1,2,71.00000,71.00000,away,1,Watford,West Ham United
2541,4071,1,3,71.00000,78.00000,both,0,Watford,West Ham United
2542,4071,1,3,78.00000,78.00000,away,1,Watford,West Ham United


# models

For match $k$  

Scoring intensities for home and away team respectively are:  

\begin{equation}
\lambda_k(t) = \lambda_k = \alpha_{i(k)}\beta_{j(k)}\gamma_h,
\end{equation}

\begin{equation}
\mu_k(t) = \mu_k = \alpha_{j(k)}\beta_{i(k)}
\end{equation}

The likelihood is:  

\begin{equation}
L(\mathbf{t}_k, \mathbf{J}_k) = \exp(-\Lambda[0, 1]) \exp(-\Upsilon[0, 1]) \prod_{l=1}^{m_k} \lambda_k(t_{k,l})^{1-J_{k,l}} \mu_k(t_{k,l})^{J_{k,l}},
\end{equation}

\begin{equation}
\Lambda[t_1, t_2] = \int_{t_1}^{t_2} \lambda_k(t) dt
\end{equation}

\begin{equation}
\Upsilon[t_1, t_2] = \int_{t_1}^{t_2} \mu_k(t) dt  
\end{equation}

where  
$J$ is indicator for home goal  
$m_k$ is the total number of goals in the match  
  
The likelihood can be broken down into the first 2 terms which is the time waiting for goals to occur. This is the time spent between the goals and the time spent after the last goal. Related to the survival function of an exponential distribution, representing the probability of scoring 0 goals.  
And the final 2 terms which are probability density of a goal being scored at time t.    

In [8]:
teams_home = np.asarray(df_goal_events.home_team.values, dtype=str, order="C")
teams_away = np.asarray(df_goal_events.away_team.values, dtype=str, order="C")

In [9]:
teams = np.sort(np.unique(np.concatenate([teams_home, teams_away])))
n_teams = len(teams)
team_to_idx = {team: i for i, team in enumerate(teams)}
home_indices = np.array([team_to_idx[t] for t in teams_home], dtype=np.int64, order="C")
away_indices = np.array([team_to_idx[t] for t in teams_away], dtype=np.int64, order="C")

In [10]:
# parameters
alphas = [0.0] * n_teams
betas = [0.0] * n_teams
constant = 0.4
home_advantage = 0.1

params = np.concatenate([alphas, betas, [constant], [home_advantage]])

In [11]:
def lambda_k(
    t: int, alpha_i: float, beta_j: float, constant: float, home_advantage: float
) -> float:
    """
    Scoring intensity for the home team in match k.
    """

    return np.exp(constant + alpha_i + beta_j + home_advantage)


def mu_k(t: int, alpha_j: float, beta_i: float, constant: float) -> float:
    """
    Scoring intensity for the away team in match k.
    """

    return np.exp(constant + alpha_j + beta_i)

In [12]:
def calculate_log_likelihood(
    # params
    alphas: list[np.float64],
    betas: list[np.float64],
    constant: float,
    home_advantage: float,
    # model functions,
    lambda_k,
    mu_k,
    # codes
    home_indices: list[int],
    away_indices: list[int],
    # obs data
    start_minutes: list[float],
    end_minutes: list[float],
    goals: list[bool],
    home_scores: list[int],
    away_scores: list[int],
    homes: list[int],
):
    num_rows = start_minutes.shape[0]

    log_likelihood = 0.0
    for row_idx in range(num_rows):
        home_idx = home_indices[row_idx]
        away_idx = away_indices[row_idx]

        alpha_i = alphas[home_idx]
        alpha_j = alphas[away_idx]

        beta_i = betas[home_idx]
        beta_j = betas[away_idx]

        log_likelihood += calculate_row_log_likelihood(
            # params
            alpha_i,
            beta_i,
            alpha_j,
            beta_j,
            constant,
            home_advantage,
            # model functions,
            lambda_k,
            mu_k,
            # obs data
            start_minutes[row_idx],
            end_minutes[row_idx],
            goals[row_idx],
            home_scores[row_idx],
            away_scores[row_idx],
            homes[row_idx],
        )

    return log_likelihood


def calculate_match_log_likelihood(
    # params
    alpha_i: float,
    beta_i: float,
    alpha_j: float,
    beta_j: float,
    constant: float,
    home_advantage: float,
    # model functions,
    lambda_k,
    mu_k,
    # obs data
    start_minutes: list[float],
    end_minutes: list[float],
    goals: list[bool],
    home_scores: list[int],
    away_scores: list[int],
    homes: list[int],
) -> float:
    """
    Log likelihood for a match between team i and team j.
    """
    log_likelihood = 0.0
    for match_row_idx in range(len(start_minutes)):
        log_likelihood += calculate_row_log_likelihood(  # params
            alpha_i,
            beta_i,
            alpha_j,
            beta_j,
            constant,
            home_advantage,
            # model functions,
            lambda_k,
            mu_k,
            # obs data
            start_minutes[match_row_idx],
            end_minutes[match_row_idx],
            goals[match_row_idx],
            home_scores[match_row_idx],
            away_scores[match_row_idx],
            homes[match_row_idx],
        )
    return log_likelihood


def calculate_row_log_likelihood(
    # params
    alpha_i: float,
    beta_i: float,
    alpha_j: float,
    beta_j: float,
    constant: float,
    home_advantage: float,
    # model functions,
    lambda_k,
    mu_k,
    # obs data
    start_minute: float,
    end_minute: float,
    goal: bool,
    home_score: int,
    away_score: int,
    home: int,
):
    if goal:
        t = start_minute
        J_k_l = home
        return calculate_goal_log_likelihood(  # params
            alpha_i,
            beta_i,
            alpha_j,
            beta_j,
            constant,
            home_advantage,
            # model functions,
            lambda_k,
            mu_k,
            # obs data
            t,
            J_k_l,
        )
    else:
        return calculate_wait_log_likelihood(
            lambda t: lambda_k(t, alpha_i, beta_j, constant, home_advantage),
            lambda t: mu_k(t, alpha_j, beta_i, constant),
            start_minute / 90.0,
            end_minute / 90.0,
        )


def calculate_goal_log_likelihood(
    # params
    alpha_i: float,
    beta_i: float,
    alpha_j: float,
    beta_j: float,
    constant,
    home_advantage: float,
    # model functions,
    lambda_k,
    mu_k,
    # obs data
    t: float,
    J_k_l: float,
) -> float:
    return np.log(
        (lambda_k(t, alpha_i, beta_j, constant, home_advantage) ** J_k_l)
        * (mu_k(t, alpha_j, beta_i, constant) ** (1 - J_k_l))
    )


def _integrate(func, a, b, n=100):
    """
    Approximates the integral of a function using the trapezoidal rule.

    Args:
        func (function): Function to integrate.
        a (float): Lower limit of integration.
        b (float): Upper limit of integration.
        n (int): Number of trapezoids.

    Returns:
        float: Approximate value of the integral.
    """
    h = (b - a) / n
    x = np.linspace(a, b, n + 1)
    y = np.vectorize(func)(x)
    return h * (0.5 * y[0] + np.sum(y[1:-1]) + 0.5 * y[-1])


def calculate_wait_log_likelihood(
    lamdba_k_of_t, mu_k_of_t, start_minute: float, end_minute: float
) -> float:
    return -_integrate(lamdba_k_of_t, start_minute, end_minute) - _integrate(
        mu_k_of_t, start_minute, end_minute
    )

In [13]:
start_minutes = np.asarray(
    df_goal_events.start_minute.values, dtype=np.float64, order="C"
)
end_minutes = np.asarray(df_goal_events.end_minute.values, dtype=np.float64, order="C")
goals = np.asarray(df_goal_events.goal.values, dtype=bool, order="C")
home_scores = np.asarray(df_goal_events.home_score.values, dtype=np.int64, order="C")
away_scores = np.asarray(df_goal_events.away_score.values, dtype=np.int64, order="C")
homes = np.asarray((df_goal_events.side == "home"), dtype=bool, order="C")

In [14]:
# i = 0

# home_idx = home_indices[i]
# away_idx = away_indices[i]

# alpha_i = alphas[home_idx]
# alpha_j = alphas[away_idx]

# beta_i = betas[home_idx]
# beta_j = betas[away_idx]

In [15]:
calculate_log_likelihood(
    # params
    alphas,
    betas,
    constant,
    home_advantage,
    # model functions,
    lambda_k,
    mu_k,
    # codes
    home_indices,
    away_indices,
    # obs data
    start_minutes,
    end_minutes,
    goals,
    home_scores,
    away_scores,
    homes,
)

-767.8183873365637

In [16]:
def loss_function(params) -> float:
    # get params
    alphas = np.asarray(params[:n_teams], dtype=np.double, order="C")
    betas = np.asarray(params[n_teams : 2 * n_teams], dtype=np.double, order="C")
    constant = params[-2]
    home_advantage = params[-1]

    return -calculate_log_likelihood(
        # params
        alphas,
        betas,
        constant,
        home_advantage,
        # model functions,
        lambda_k,
        mu_k,
        # codes
        home_indices,
        away_indices,
        # obs data
        start_minutes,
        end_minutes,
        goals,
        home_scores,
        away_scores,
        homes,
    )

In [17]:
constraints = [
    {"type": "eq", "fun": lambda x: sum(x[:n_teams])},
    {"type": "eq", "fun": lambda x: sum(x[n_teams : 2 * n_teams])},
]
bounds = [(-2, 2)] * (2 * n_teams) + [(-2, 1)] + [(-0.5, 1.0)]
options = {
    "maxiter": 100,
    "disp": True,
    "ftol": 1e-14,  # Very strict function tolerance
    # "eps": 1e-12,  # Very small step size
}

iteration = 0


def callback_function(xk, *args):
    # xk is the current parameter vector
    # Print the current iteration values
    global iteration
    iteration += 1
    current_loss = loss_function(xk)
    print(f"Iteration {iteration}: Loss = {current_loss}")

    # You could also print specific parameter values if needed
    # print(f"Current alpha values: {xk[:n_teams]}")
    # print(f"Current beta values: {xk[n_teams:2*n_teams]}")
    # print(f"Current home advantage: {xk[-1]}")

    return False  # Return False to continue optimization


res = scipy.optimize.minimize(
    loss_function,
    params,
    constraints=constraints,
    bounds=bounds,
    options=options,
    callback=callback_function,
    # method="L-BFGS-B",
    # method="trust-constr",
    # method="SLSQP",
    # method="COBYQA",
)

Iteration 1: Loss = 2461.3027426712533
Iteration 2: Loss = 32797.79024324296
Iteration 3: Loss = 6105.673668688497
Iteration 4: Loss = 15138.441876459388
Iteration 5: Loss = 2417.4092929922012
Iteration 6: Loss = 7480.507057287203
Iteration 7: Loss = 2988.123332866773
Iteration 8: Loss = 1609.663509767134
Iteration 9: Loss = 1206.6229565628937
Iteration 10: Loss = 810.4173091181292
Iteration 11: Loss = 934.1849602371992
Iteration 12: Loss = 701.8444331238188
Iteration 13: Loss = 660.1603595644245
Iteration 14: Loss = 731.1070453659066
Iteration 15: Loss = 684.0276994805444
Iteration 16: Loss = 718.7150909448114
Iteration 17: Loss = 663.550698283551
Iteration 18: Loss = 648.294244408735
Iteration 19: Loss = 647.8146680349366
Iteration 20: Loss = 660.2712654334731
Iteration 21: Loss = 656.9188793925665
Iteration 22: Loss = 648.4113341030826
Iteration 23: Loss = 646.6692589059423
Iteration 24: Loss = 646.5751718911912
Iteration 25: Loss = 646.7697733192723
Iteration 26: Loss = 646.6260755

In [18]:
res

 message: Iteration limit reached
 success: False
  status: 9
     fun: 646.5719944939248
       x: [ 4.138e-01  1.327e-01 ...  7.237e-02  2.238e-01]
     nit: 100
     jac: [-3.052e-05  0.000e+00 ... -9.155e-05 -7.629e-05]
    nfev: 4676
    njev: 100

In [19]:
res["x"]

array([ 0.41384358,  0.13266247, -0.3691286 , -0.13126401, -0.39918559,
        0.15288693, -0.05349165,  0.05330923, -0.34405233, -0.79458142,
       -0.00583899,  0.52691074,  0.63836937,  0.25846757, -0.23579077,
       -0.16792734,  0.31518071,  0.01918305,  0.05581385, -0.0653668 ,
        0.07237552,  0.35168205,  0.14730171,  0.25258527,  0.29765605,
       -0.3082659 , -0.00473132, -0.14559427,  0.47353999,  0.40070692,
       -0.05844943, -0.86396694, -0.75214233,  0.06097279, -0.08697055,
        0.18636614, -0.20311127,  0.1554606 ,  0.09945046, -0.0748655 ,
        0.07236772,  0.22379169])

In [20]:
np.exp(res["x"])

array([1.51262051, 1.14186452, 0.6913365 , 0.87698621, 0.67086618,
       1.16519323, 0.94791386, 1.05475576, 0.70889183, 0.4517703 ,
       0.99417802, 1.69369196, 1.89339093, 1.29494416, 0.78994593,
       0.84541526, 1.37050695, 1.01936823, 1.05740083, 0.93672381,
       1.07505897, 1.4214565 , 1.15870351, 1.28734927, 1.34669851,
       0.73471993, 0.99527986, 0.86450839, 1.6056682 , 1.49287967,
       0.94322594, 0.42148675, 0.47135567, 1.06286999, 0.91670409,
       1.20486332, 0.81618742, 1.16819591, 1.10456375, 0.92786827,
       1.07505059, 1.25081043])

In [21]:
params = dict(
    zip(
        ["constant", "home_advantage"],
        res["x"][-2:],
    )
)
params

{'constant': 0.07236772091428051, 'home_advantage': 0.2237916852344791}

In [22]:
pd.DataFrame(
    {
        "team": teams,
        "alpha": res["x"][:n_teams],
        "beta": res["x"][n_teams : 2 * n_teams],
    }
).sort_values("alpha", ascending=False, ignore_index=True)

,team,alpha,beta
0,Manchester City,0.63837,-0.75214
1,Liverpool,0.52691,-0.86397
2,Arsenal,0.41384,0.07238
3,Tottenham Hotspur,0.31518,-0.20311
4,Manchester United,0.25847,0.06097
5,Chelsea,0.15289,-0.30827
6,Bournemouth,0.13266,0.35168
7,West Ham United,0.05581,0.09945
8,Everton,0.05331,-0.14559
9,Watford,0.01918,0.15546


In [23]:
sum([params["attack_" + team] for team in teams])

KeyError: 'attack_Arsenal'

In [ ]:
sum([params["defence_" + team] for team in teams])

In [ ]:
alphas_fit = res["x"][:n_teams]
betas_fit = res["x"][n_teams : 2 * n_teams]
constant_fit = res["x"][-2]
home_advantage_fit = res["x"][-1]

In [ ]:
calculate_log_likelihood(
    # params
    alphas_fit,
    betas_fit,
    constant_fit,
    home_advantage_fit,
    # model functions,
    lambda_k,
    mu_k,
    # codes
    home_indices,
    away_indices,
    # obs data
    start_minutes,
    end_minutes,
    goals,
    home_scores,
    away_scores,
    homes,
)

In [ ]:
constant_fit_edited = constant_fit
# constant_fit_edited = 0.2
home_advantage_fit_edited = home_advantage_fit
home_advantage_fit_edited = -0.0

In [ ]:
calculate_log_likelihood(
    # params
    alphas_fit,
    betas_fit,
    constant_fit_edited,
    home_advantage_fit_edited,
    # model functions,
    lambda_k,
    mu_k,
    # codes
    home_indices,
    away_indices,
    # obs data
    start_minutes,
    end_minutes,
    goals,
    home_scores,
    away_scores,
    homes,
)

In [ ]:
alpha_i = 0.0
beta_i = 0.0
alpha_j = 0.0
beta_j = 0.0
constant = 0.0
home_advantage = 0.0
# obs data
t = 1
J_k_l = 1

In [ ]:
calculate_goal_log_likelihood(
    alpha_i,
    beta_i,
    alpha_j,
    beta_j,
    constant,
    home_advantage,
    # model functions,
    lambda_k,
    mu_k,
    # obs data
    t,
    J_k_l,
)

In [ ]:
lamdba_k_of_t = lambda t: lambda_k(t, alpha_i, beta_j, constant, home_advantage)
mu_k_of_t = lambda t: mu_k(t, alpha_j, beta_i, constant)
start_minute = 0
end_minute = 90 / 90

In [ ]:
calculate_wait_log_likelihood(lamdba_k_of_t, mu_k_of_t, start_minute, end_minute)